# Replications of the Experiments in RNSB paper

In [3]:
from wefe.datasets import load_bingliu
from wefe.metrics import RNSB
from wefe.query import Query
from wefe.word_embedding_model import WordEmbeddingModel

import pandas as pd
import plotly.express as px
import gensim.downloader as api

# load the target word sets.
# In this case each word is an objective set because each of them represents a different social group.
RNSB_words = [
    ['swedish'], ['irish'], ['mexican'], ['chinese'], ['filipino'], ['german'], ['english'],
    ['french'], ['norwegian'], ['american'], ['indian'], ['dutch'], ['russian'],
    ['scottish'], ['italian']
]

bing_liu = load_bingliu()

# Create the query
query = Query(RNSB_words,
              [bing_liu['positive_words'], bing_liu['negative_words']])

# Fetch the models
glove = WordEmbeddingModel(api.load('glove-wiki-gigaword-300'),
                           'glove-wiki-gigaword-300')
# note that conceptnet uses a /c/en/ prefix before each word.
conceptnet = WordEmbeddingModel(api.load('conceptnet-numberbatch-17-06-300'),
                                'conceptnet-numberbatch-17',
                                vocab_prefix='/c/en/')

# Run the queries
glove_results = RNSB().run_query(query, glove)
conceptnet_results = RNSB().run_query(query, conceptnet)

In [4]:
# Show the results obtained with glove
glove_fig = px.bar(
    pd.DataFrame(glove_results['negative_sentiment_distribution'].items(),
                 columns=['Word', 'Sentiment distribution']), x='Word',
    y='Sentiment distribution', title='Glove negative sentiment distribution')
glove_fig.update_yaxes(range=[0, 0.2])
glove_fig.show()


In [4]:
# Show the results obtained with conceptnet
conceptnet_fig = px.bar(
    pd.DataFrame(conceptnet_results['negative_sentiment_distribution'].items(),
                 columns=['Word', 'Sentiment distribution']), x='Word',
    y='Sentiment distribution',
    title='Conceptnet negative sentiment distribution')
conceptnet_fig.update_yaxes(range=[0, 0.2])
conceptnet_fig.show()

In [5]:
# Show a fairness distribution.
fair_distribution = pd.DataFrame(
    conceptnet_results['negative_sentiment_distribution'].items(),
    columns=['Word', 'Sentiment distribution'])
fair_distribution['Sentiment distribution'] = np.ones(
    fair_distribution.shape[0]) / fair_distribution.shape[0]

fair_distribution_fig = px.bar(fair_distribution, x='Word',
                               y='Sentiment distribution',
                               title='Fair negative sentiment distribution')
fair_distribution_fig.update_yaxes(range=[0, 0.2])
fair_distribution_fig.show()